In [ ]:
import os
import tarfile
import urllib
import pandas as pd

DOWNLOAD_ROOT = 'https://raw.githubusercontent.com/ageron/handson-ml2/master/'
HOUSING_PATH = os.path.join('datasets', 'housing')
HOUSING_URL = f'{DOWNLOAD_ROOT}datasets/housing/housing.tgz'

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    os.makedirs(housing_path, exist_ok=True)
    tgz_path = os.path.join(housing_path, 'housing.tgz')
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, 'housing.csv')
    return pd.read_csv(csv_path)

In [ ]:
fetch_housing_data()
housing = load_housing_data()
housing.head()

In [ ]:
housing.info()

In [ ]:
housing['ocean_proximity'].value_counts()

In [ ]:
housing.describe()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
housing.hist(bins=50, figsize=(20,15))
plt.show()

In [ ]:
import numpy as np

def split_train_test(data, ratio):
    # Set the random seed to ensure consistent shuffled indices
    np.random.seed(42)

    # Generate array of len(data) 'random' integers
    shuffled_i = np.random.permutation(len(data))
    
    # Establish size of test set as a portion of len(data)
    test_set_size = int(len(data) * ratio)
    
    # Test indices make up the one portion, training indeces make up the rest
    test_i = shuffled_i[:test_set_size]
    train_i = shuffled_i[test_set_size:]
    
    return data.iloc[train_i], data.iloc[test_i]

In [ ]:
# Set aside 20% of the data for a test set
train_set, test_set = split_train_test(housing, 0.2)
print(f'Train: {len(train_set)}\nTest: {len(test_set)}')

In [ ]:
from zlib import crc32

def test_set_check(identifier, ratio):
    return crc32(np.int64(identifier)) & 0xffffffff < ratio * 2**32

def split_train_test_by_id(data, ratio, id_column):
    ids = data[id_column]
    
    # Split values by ID checksum less than or equal to the ratio threshold
    in_test_set = ids.apply(lambda id_: test_set_check(id_, ratio))
    return data.loc[~in_test_set], data.loc[in_test_set]

In [ ]:
# We choose to use an index column for ID as they are unique
housing_with_id = housing.reset_index()
train_set, test_set = split_train_test_by_id(housing_with_id, 0.2, 'index')

In [ ]:
#train_set.head()
test_set.head()

In [ ]:
# We could also use lat and long to ensure unique values that don't rely on order
housing_with_id['id'] = housing['longitude'] * 1000 + housing['latitude']
train_set, test_set = split_train_test_by_id(housing_with_id, 0.2, 'id')

In [ ]:
#train_set.head()
test_set.head()

In [ ]:
# Scikit-Learn has similar functions for splitting data
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

In [ ]:
#train_set.head()
test_set.head()

In [ ]:
# Categorize incomes to ensure the test set represents the full set of data
housing['income_cat'] = pd.cut(
    housing['median_income'],
    bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
    labels=[1, 2, 3, 4, 5]
)
housing['income_cat'].hist()

In [ ]:
# Use Scikit-Learn's StratifiedShuffleSplit to split housing according to the income categories
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_i, test_i in split.split(housing, housing['income_cat']):
    strat_train_set = housing.loc[train_i]
    strat_test_set = housing.loc[test_i]

In [ ]:
#strat_train_set.head()
strat_test_set.head()

In [ ]:
# Verify by checking the percentage of the test set that falls in each category
strat_test_set['income_cat'].value_counts() / len(strat_test_set)

In [ ]:
def income_cat_proportions(data):
    return data['income_cat'].value_counts() / len(data)

train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

compare_props = pd.DataFrame({
    'Overall': income_cat_proportions(housing),
    'Stratified': income_cat_proportions(strat_test_set),
    'Random': income_cat_proportions(test_set)
}).sort_index()

compare_props['Rand. %error'] = compare_props['Random'] / compare_props['Overall'] * 100 - 100
compare_props['Strat. %error'] = compare_props['Stratified'] / compare_props['Overall'] * 100 - 100

compare_props

In [ ]:
for set_ in (strat_train_set, strat_test_set):
    set_.drop('income_cat', axis=1, inplace=True)

In [ ]:
#strat_train_set.head()
strat_test_set.head()

In [ ]:
# Copy the training set into an exploration set
housing = strat_train_set.copy()

In [ ]:
# Visualize the data by geographical location
housing.plot(kind='scatter', x='longitude', y='latitude', alpha=0.1)

In [ ]:
# Add median house value to the visualization
housing.plot(
    kind='scatter', 
    x='longitude', 
    y='latitude', 
    alpha=0.4,
    s=housing['population']/100,
    label='population',
    figsize=(10,7),
    c='median_house_value',
    cmap=plt.get_cmap('jet'),
    colorbar=True
)
plt.legend()

In [ ]:
# Calculate the standard correlation coefficients (Pearson's r)
corr_matrix = housing.corr()
corr_matrix['median_house_value'].sort_values(ascending=False)

In [ ]:
from pandas.plotting import scatter_matrix

# Select 4 most highly correlated attributes and generate a scatter matrix between them
attributes = corr_matrix['median_house_value'].sort_values(ascending=False).keys()[0:4]
scatter_matrix(housing[attributes], figsize=(12, 8))

In [ ]:
# Check the median_income vs median_house_value plot; the horizontalish lines indicate potential anomalies
housing.plot(kind='scatter', x='median_income', y='median_house_value', alpha=0.1)

In [ ]:
# Create some additional attributes we may be interested in
housing['rooms_per_household'] = housing['total_rooms']/housing['households']
housing['bedrooms_per_room'] = housing['total_bedrooms']/housing['total_rooms']
housing['population_per_household'] = housing['population']/housing['households']

In [ ]:
# Get new correlations
corr_matrix = housing.corr()
corr_matrix['median_house_value'].sort_values(ascending=False)

In [ ]:
# Reset to a clean copy of the data, separating predictors and labels for their respective transformations
housing = strat_train_set.drop('median_house_value', axis=1)
housing_labels = strat_train_set['median_house_value'].copy()

In [ ]:
# Address missing values in total_bedrooms

# Option 1: Remove districts missing this attribute
#housing.dropna(subset=['total_bedrooms'])

# Option 2: Remove the entire column
#housing.drop('total_bedrooms', axis=1)

# Option 3: Fill missing values (median in this case)
#median = housing['total_bedrooms'].median()
#housing['total_bedrooms'].fillna(median, inplace=True)

In [ ]:
# We can use Scikit Learn to do the same for the entire dataset, but only if it's entirely numeric
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='median')

In [ ]:
# Since ocean_proximity is the only non-numeric attribute, we make a copy of the data excluding only that
housing_numeric = housing.drop('ocean_proximity', axis=1)

In [ ]:
# Get rows missing total_bedrooms
missing_total_bedrooms = housing_numeric[np.isnan(housing_numeric['total_bedrooms'])].index
housing_numeric.loc[missing_total_bedrooms].head()

In [ ]:
# The Imputer computes and stores the median of each attribute, useful if we encounter missing data later
#imputer.fit(housing_numeric)

# Apply the transformation with the Imputer to generate a NumPy array
#X = imputer.transform(housing_numeric)
X = imputer.fit_transform(housing_numeric)

print(f'Imputer:\n{imputer.statistics_}\n\nPandas:\n{housing_numeric.median().values}')

In [ ]:
# Convert the NumPy array back into a dataframe
housing_transformed = pd.DataFrame(X, columns=housing_numeric.columns, index=housing_numeric.index)
housing_transformed.loc[missing_total_bedrooms].head()

In [ ]:
# Ocean_proximity was dropped from our numerical set, here it is added to its own dataframe
housing_cat = housing[['ocean_proximity']]
housing_cat.head(10)

In [ ]:
# Use Scikit Learn's Ordinal Encoder to encode ocean_proximity into numerical values
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)
housing_cat_encoded[:10]

In [ ]:
# Access the enumerated categories by calling the hyperparameter categories_
ordinal_encoder.categories_

In [ ]:
# Apply one-hot encoding to 
from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder()
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot.toarray()

In [ ]:
# Categories accessed the same as for ordinal_encoder
cat_encoder.categories_

In [ ]:
# Transformer class to add new attributes from earlier
from sklearn.base import BaseEstimator, TransformerMixin

rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)
#housing_extra_attribs

In [ ]:
# Use Scikit Learn's Pipeline class to sequence the transformations
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Create the pipeline for numerical data
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('attribs_adder', CombinedAttributesAdder()),
    ('std_scaler', StandardScaler())
])

housing_num_tr = num_pipeline.fit_transform(housing_numeric)
#housing_num_tr

In [ ]:
# ColumnTransformer can apply different encoders to specified DataFrame columns
from sklearn.compose import ColumnTransformer

# Separate the columns by numeric and categorical; indices would work as well
num_attribs = list(housing_numeric)
cat_attribs = ['ocean_proximity']
#num_attribs = [housing.columns.get_loc(c) for c in list(housing_numeric) if c in housing]
#cat_attribs = [housing.columns.get_loc('ocean_proximity')]

# Use the numerical pipeline for the numerical attributes, one-hot for categorical
full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', OneHotEncoder(), cat_attribs)
])

housing_prepared = full_pipeline.fit_transform(housing)
housing_prepared

In [ ]:
# Prepare a Linear Regression model
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

In [ ]:
# Test the model with a small sample of the data
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)
print(f'Predictions: {lin_reg.predict(some_data_prepared)}')
print(f'Labels: {list(some_labels)}')

In [ ]:
# Check the accuracy using the full dataset
from sklearn.metrics import mean_squared_error

housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)

print(f'MSE: {lin_mse}\nRMSE: {lin_rmse}')

In [ ]:
# Try a Decision Tree to improve the model accuracy
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, housing_labels)

In [ ]:
# Check the accuracy of the new model
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)

print(f'MSE: {tree_mse}\nRMSE: {tree_rmse}')

In [ ]:
# Validate the model using Scikit Learn's K-fold cross-validation feature
from sklearn.model_selection import cross_val_score

# cross_val_score expects a utility function (as opposed to a cost function), hence neg_mean_squared_error
scores = cross_val_score(
    tree_reg,
    housing_prepared,
    housing_labels,
    scoring='neg_mean_squared_error',
    cv=10
)

tree_rmse_scores = np.sqrt(-scores)

In [ ]:
# Helper function for printing scores
def display_scores(scores):
    print(f'Scores: {scores}\nMean: {scores.mean()}\nStandard Deviation: {scores.std()}')

display_scores(tree_rmse_scores)

In [ ]:
# Cross-validate the Linear Regression model
lin_scores = cross_val_score(
    lin_reg,
    housing_prepared,
    housing_labels,
    scoring='neg_mean_squared_error',
    cv=10
)

lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

In [ ]:
# Try one more model (Random Forest Regressor)
#from sklearn.ensemble import RandomForestRegressor

#forest_reg = RandomForestRegressor()
#forest_reg.fit(housing_prepared, housing_labels)

In [ ]:
"""
# Validate once more
forest_scores = cross_val_score(
    forest_reg,
    housing_prepared,
    housing_labels,
    scoring='neg_mean_squared_error',
    cv=10
)

forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)
"""

In [ ]:
# Use grid search to tune hyperparameters
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid = [
    {'n_estimators': [30, 100, 300], 'max_features': [.25, .5, .75, 1.0]},
    {'bootstrap': [False], 'n_estimators': [30, 100], 'max_features': [.5, .75, 1.0]}
]

forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(
    forest_reg,
    param_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    return_train_score=True
)

grid_search.fit(housing_prepared, housing_labels)

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_estimator_

In [ ]:
cv_results = grid_search.cv_results_
for mean_score, params in zip(cv_results['mean_test_score'], cv_results['params']):
    print(np.sqrt(-mean_score), params)

In [ ]:
# Determine and display importance of each feature
feature_importances = grid_search.best_estimator_.feature_importances_
extra_attribs = ['rooms_per_hhold', 'pop_per_hhold', 'bedrooms_per_room']
cat_encoder = full_pipeline.named_transformers_['cat']
cat_one_hot_attribs = list(cat_encoder.categories_[0])
attributes = num_attribs + extra_attribs + cat_one_hot_attribs
sorted(zip(feature_importances, attributes), reverse=True)

In [ ]:
# Evaluate the model performance with the test set
final_model =  grid_search.best_estimator_
X_test = strat_test_set.drop('median_house_value', axis=1)
y_test = strat_test_set['median_house_value'].copy()

X_test_prepared = full_pipeline.transform(X_test)

final_predictions = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)

final_rmse

In [ ]:
# Compute the 95% confidence interval of the results
from scipy import stats

In [ ]:
confidence = 0.95
squared_errors = (final_predictions - y_test) ** 2
np.sqrt(
    stats.t.interval(
        confidence,
        len(squared_errors) - 1,
        loc=squared_errors.mean(),
        scale=stats.sem(squared_errors)
    )
)

# Exercises

1) Try a Support Vector Machine regressor (sklearn.svm.SVR), with various hyperparameters such as kernel="linear" (with various values for the C hyperparameter) or kernel="rbf" (with various values for the C and gamma hyperparameters). Don't worry about what these hyperparameters mean for now. How does the best SVR predictor perform?

In [ ]:
# Helper function for cross-validation of various models
def cross_validate(model):
    scores = cross_val_score(
        model,
        housing_prepared,
        housing_labels,
        scoring='neg_mean_squared_error',
        cv=10
    )
    rmse_scores = np.sqrt(-scores)
    display_scores(rmse_scores)

In [ ]:
# Support Vector Machines
from sklearn.svm import SVR

In [ ]:
# Linear kernel
#linear_svm = SVR(kernel='linear')
#linear_svm.fit(housing_prepared, housing_labels)

In [ ]:
#cross_validate(linear_svm)

In [ ]:
# Gaussian kernel
#rbf_svm = SVR(kernel='rbf')
#rbf_svm.fit(housing_prepared, housing_labels)

In [ ]:
#cross_validate(rbf_svm)

In [ ]:
"""
svm_params = [
    {
        'kernel': ['linear'], 
        'C': [10., 30., 100., 300., 1000., 3000., 10000., 30000.]
    },{
        'kernel': ['rbf'], 
        'C': [10., 30., 100., 300., 1000., 3000.],
        'gamma': [0.01, 0.03, 0.1, 0.3, 1.0, 3.0]
    }
]

svm_reg = SVR()

grid_search = GridSearchCV(
    svm_reg,
    svm_params,
    cv=5,
    scoring='neg_mean_squared_error',
    return_train_score=True
)

grid_search.fit(housing_prepared, housing_labels)
"""

In [ ]:
# Helper function to get RMSE using best hyperparameters from search results
def test_best_hyperparameters(search_results):
    # Evaluate the model performance with the test set
    final_model =  search_results.best_estimator_
    X_test = strat_test_set.drop('median_house_value', axis=1)
    y_test = strat_test_set['median_house_value'].copy()

    X_test_prepared = full_pipeline.transform(X_test)

    final_predictions = final_model.predict(X_test_prepared)

    squared_errors = (final_predictions - y_test) ** 2
    rmse = np.sqrt(
        stats.t.interval(
            0.95,
            len(squared_errors) - 1,
            loc=squared_errors.mean(),
            scale=stats.sem(squared_errors)
        )
    )

    print(f'Best hyperparameters: {search_results.best_params_}\nRMSE: {rmse}')

In [ ]:
test_best_hyperparameters(grid_search)

2) Try replacing GridSearchCV with RandomizedSearchCV.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import expon, reciprocal

In [ ]:
"""
param_distribs = [
    {
        'kernel': ['linear', 'rbf'],
        'C': reciprocal(20, 200000),
        'gamma': expon(scale=1.0)
    }
]

svm_reg = SVR()

random_search = RandomizedSearchCV(
    svm_reg,
    param_distributions=param_distribs,
    n_iter=50,
    cv=5,
    scoring='neg_mean_squared_error',
    return_train_score=True,
    random_state=42
)

random_search.fit(housing_prepared, housing_labels)
"""

In [ ]:
test_best_hyperparameters(random_search)

3) Try adding a transformer in the preparation pipeline to select only the most important attributes.

In [62]:
import numpy as np

In [63]:
# Recall feature_importances calculated with the RandomForestRegressor grid search
feature_importances = [
    (0.3758106238507585, 'median_income'),
    (0.1531916154269434, 'INLAND'),
    (0.11316103307394935, 'pop_per_hhold'),
    (0.07380408048739615, 'longitude'),
    (0.06846831115043858, 'latitude'),
    (0.053333614007285805, 'rooms_per_hhold'),
    (0.050694287855152495, 'bedrooms_per_room'),
    (0.04175094704179171, 'housing_median_age'),
    (0.014694621081341083, 'total_rooms'),
    (0.014257664082623828, 'population'),
    (0.014015238382847444, 'total_bedrooms'),
    (0.013770060742181808, 'households'),
    (0.0069693152264157215, '<1H OCEAN'),
    (0.004249300120337744, 'NEAR OCEAN'),
    (0.0017753375854791455, 'NEAR BAY'),
    (5.3949885057266795e-05, 'ISLAND')
]

# Split feature importances into two lists 
importances, attributes = zip(*feature_importances)

# Get sorted indices of top k elements in a list
def indices_of_top_k(a, k):
    b = np.argpartition(np.array(a), -k)[-k:]
    return np.sort(b)

top_k = indices_of_top_k(importances, 5)
np.array(attributes)[top_k]

array(['median_income', 'INLAND', 'pop_per_hhold', 'longitude',
       'latitude'], dtype='<U18')

In [64]:
# Create a new transformer to select only the indices determined by indices_of_top_k
from sklearn.base import BaseEstimator, TransformerMixin

class ImportantAttributeSelector(BaseEstimator, TransformerMixin):
    def __init__(self, importances, k):
        self.importances = importances
        self.k = k
        
    def fit(self, X, y=None):
        self.indices_ = indices_of_top_k(self.importances, self.k)
        return self
    
    def transform(self, X):
        return X[:, self.indices_]

In [67]:
top_attributes_pipeline = Pipeline([
    ('preparation', full_pipeline),
    ('top_attributes', ImportantAttributeSelector(importances, 5))
])

In [68]:
housing_prepared_top_k = top_attributes_pipeline.fit_transform(housing)

In [72]:
housing_prepared_top_k[0:5]

array([[-1.15604281,  0.77194962,  0.74333089, -0.49323393, -0.44543821],
       [-1.17602483,  0.6596948 , -1.1653172 , -0.90896655, -1.0369278 ],
       [ 1.18684903, -1.34218285,  0.18664186, -0.31365989, -0.15334458],
       [-0.01706767,  0.31357576, -0.29052016, -0.36276217, -0.39675594],
       [ 0.49247384, -0.65929936, -0.92673619,  1.85619316,  2.41221109]])

In [73]:
housing_prepared[0:5, top_k]

array([[-1.15604281,  0.77194962,  0.74333089, -0.49323393, -0.44543821],
       [-1.17602483,  0.6596948 , -1.1653172 , -0.90896655, -1.0369278 ],
       [ 1.18684903, -1.34218285,  0.18664186, -0.31365989, -0.15334458],
       [-0.01706767,  0.31357576, -0.29052016, -0.36276217, -0.39675594],
       [ 0.49247384, -0.65929936, -0.92673619,  1.85619316,  2.41221109]])

4) Try creating a single pipeline that does the full data preparation plus the final prediction.

In [ ]:
prediction_pipeline = Pipeline([
    ('preparation', full_pipeline),
    ('top_attributes', ImportantAttributeSelector(importances, 5)),
    ('svm_reg', SVR(**grid_search.best_estimator_)) # Best results from Exercise 1
])

prediction_pipeline.fit_transform(housing, housing_labels)

In [ ]:
data = prediction_pipeline.predict(housing[:5])
labels = housing_labels.iloc[:5]

print(f'Predictions: {data}\nLabels: {labels}\n')

5) Automatically explore some preparation options using GridSearchCV.

In [ ]:
param_grid = [
    {
        'preparation__num__imputer__strategy': ['mean', 'median', 'most_frequent'],
        'feature_selection__k': list(range(1, len(feature_importances) + 1))
    }
]

grid_search_prep = GridSearchCV(
    prepare_select_and_predict_pipeline, 
    param_grid, 
    cv=5,
    scoring='neg_mean_squared_error', 
    verbose=2
)

grid_search_prep.fit(housing, housing_labels)

In [ ]:
grid_search_prep.best_params_